# Pre-built cadCAD Models

This notebook shows how a data scientist can take cadCAD models "off the rack" from an MSML dev without ever needing to get into the details.

In [2]:
import sys
import os

sys.path.append(os.path.abspath('..'))

from cadCAD import cadCAD_models